## Summing all the prime numbers below a given number

In [1]:
import os
from time import perf_counter

# Parámetros (minúsculas en Python)
number = int(os.environ.get("NUMBER", "2500000"))  # default si no hay SLURM
nprocs = int(os.environ.get("NPROCS", str(os.cpu_count() or 1)))
print(f"number = {number} | nprocs = {nprocs}")

number = 2500000 | nprocs = 12


In [2]:
import time

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

# number = 2_500_000
suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

The prime sum below  2500000 is  219697708195  and the time taken is 4.986082156499227
The prime sum below  2500000 is  219697708195  and the time taken is 4.83 s ± 34.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [3]:
tiempo = %timeit -r 2 -o -q sum_primes(number)
ssuma = sum_primes(number)

3) Numba secuencial (@njit)

In [4]:
import time
from numba import njit

@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes_numba(n):
    res = 0
    for i in range(n):
        res += if_prime_numba(i)
    return res

# Warmup para compilar
_ = sum_primes_numba(10000)

start = time.time()
suma = sum_primes_numba(number)
stop = time.time()

print("Numba secuencial → sum =", suma, "tiempo =", (stop - start))

Numba secuencial → sum = 219697708195 tiempo = 0.18926239013671875


4) multiprocessing.Pool usando nprocs

In [5]:
import time
from multiprocessing import Pool

def sum_primes_pool(n, nprocs):
    with Pool(processes=nprocs) as p:
        # chunksize ayuda a reducir overhead
        res = p.map(if_prime, range(n), chunksize=5000)
    return sum(res)

start = time.time()
suma = sum_primes_pool(number, nprocs)
stop = time.time()

print(f"Pool({nprocs}) → sum = {suma} | tiempo = {stop - start:.4f} s")

Pool(12) → sum = 219697708195 | tiempo = 0.9868 s


5) Numba paralela con prange

In [6]:
import time
from numba import njit, prange, set_num_threads, get_num_threads

@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit(parallel=True)
def sum_primes_numba_parallel(n):
    s = 0
    for i in prange(n):
        s += if_prime_numba(i)
    return s

# Ajusta hilos (si tienes nprocs definido)
# set_num_threads(nprocs)
# print("Numba threads:", get_num_threads())

# Warmup (compila)
_ = sum_primes_numba_parallel(10000)

start = time.time()
suma = sum_primes_numba_parallel(number)   # number debe estar definido antes
stop = time.time()

print("Numba parallel (prange) → sum =", suma, "tiempo =", (stop - start))

Numba parallel (prange) → sum = 219697708195 tiempo = 0.03267097473144531


Celda de resulados

## 3.2.e) Resultados en *mendel* (primes) — alumno05

Ejecutado en **mendel** con `sbatch submit_Primes_mendel-alumno05.sb` (notebook **primes-par-alumno05**), para number=10^6 y 10^7 y con 1/2/4/8 cores:

```text
===============================================
number=1000000 | step_cores=-c 1
===============================================
number = 1000000 | nprocs = 1
The prime sum below  1000000 is  37550402023  and the time taken is 2.296719948450724
The prime sum below  1000000 is  37550402023  and the time taken is 2.26 s ± 20.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Numba secuencial → sum = 37550402023 tiempo = 0.10120797157287598
Pool(1) → sum = 37550402023 | tiempo = 2.3020 s
Numba parallel (prange) → sum = 37550402023 tiempo = 0.01355290412902832
===============================================
number=1000000 | step_cores=-c 2
===============================================
number = 1000000 | nprocs = 2
The prime sum below  1000000 is  37550402023  and the time taken is 2.24898091952006
The prime sum below  1000000 is  37550402023  and the time taken is 2.19 s ± 648 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Numba secuencial → sum = 37550402023 tiempo = 0.10122895240783691
Pool(2) → sum = 37550402023 | tiempo = 1.1557 s
Numba parallel (prange) → sum = 37550402023 tiempo = 0.010890007019042969
===============================================
number=1000000 | step_cores=-c 4
===============================================
number = 1000000 | nprocs = 4
The prime sum below  1000000 is  37550402023  and the time taken is 2.2537542978922525
The prime sum below  1000000 is  37550402023  and the time taken is 2.22 s ± 29.5 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Numba secuencial → sum = 37550402023 tiempo = 0.10116195678710938
Pool(4) → sum = 37550402023 | tiempo = 0.5976 s
Numba parallel (prange) → sum = 37550402023 tiempo = 0.015377521514892578
===============================================
number=1000000 | step_cores=-c 8
===============================================
number = 1000000 | nprocs = 8
The prime sum below  1000000 is  37550402023  and the time taken is 2.255213499069214
The prime sum below  1000000 is  37550402023  and the time taken is 2.19 s ± 13.4 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Numba secuencial → sum = 37550402023 tiempo = 0.10121417045593262
Pool(8) → sum = 37550402023 | tiempo = 0.3308 s
Numba parallel (prange) → sum = 37550402023 tiempo = 0.016051054000854492
===============================================
number=10000000 | step_cores=-c 1
===============================================
number = 10000000 | nprocs = 1
The prime sum below  10000000 is  3203324994356  and the time taken is 59.66280802090963
The prime sum below  10000000 is  3203324994356  and the time taken is 57.7 s ± 14.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Numba secuencial → sum = 3203324994356 tiempo = 2.537010669708252
Pool(1) → sum = 3203324994356 | tiempo = 59.9169 s
Numba parallel (prange) → sum = 3203324994356 tiempo = 0.1746387481689453
===============================================
number=10000000 | step_cores=-c 2
===============================================
number = 10000000 | nprocs = 2
The prime sum below  10000000 is  3203324994356  and the time taken is 59.28530343373617
The prime sum below  10000000 is  3203324994356  and the time taken is 57.3 s ± 16.5 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Numba secuencial → sum = 3203324994356 tiempo = 2.5353481769561768
Pool(2) → sum = 3203324994356 | tiempo = 29.8413 s
Numba parallel (prange) → sum = 3203324994356 tiempo = 0.15274930000305176
===============================================
number=10000000 | step_cores=-c 4
===============================================
number = 10000000 | nprocs = 4
The prime sum below  10000000 is  3203324994356  and the time taken is 59.2501745223999
The prime sum below  10000000 is  3203324994356  and the time taken is 57.3 s ± 7.37 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Numba secuencial → sum = 3203324994356 tiempo = 2.5365588665008545
Pool(4) → sum = 3203324994356 | tiempo = 15.2643 s
Numba parallel (prange) → sum = 3203324994356 tiempo = 0.15757036209106445
===============================================
number=10000000 | step_cores=-c 8
===============================================
number = 10000000 | nprocs = 8
The prime sum below  10000000 is  3203324994356  and the time taken is 59.09812879562378
The prime sum below  10000000 is  3203324994356  and the time taken is 57.9 s ± 6.2 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Numba secuencial → sum = 3203324994356 tiempo = 2.5368690490722656
Pool(8) → sum = 3203324994356 | tiempo = 8.2479 s
Numba parallel (prange) → sum = 3203324994356 tiempo = 0.14596796035766602
END=Tue Dec 16 13:21:49 CET 2025
```

### Comentario y análisis

- En el código original se ve que el tiempo no mejora al cambiar `-c`: se queda sobre ~2.2 s (10^6) y ~59 s (10^7). Esto tiene sentido porque esa parte realmente no usa varios cores, aunque el job los tenga reservados.
- Numba en secuencial mejora muchísimo (solo con `@njit`): para 10^6 pasa a ~0.10 s y para 10^7 a ~2.5 s. Se nota que la mayor ganancia viene de compilar y quitar la sobrecarga de Python.
- Con `multiprocessing.Pool` sí hay paralelismo real: para 10^7 baja de ~60 s a ~8.25 s con 8 procesos (y en 10^6 baja de ~2.3 s a ~0.33 s). No llega perfecto a 8×, pero el descenso es bastante claro.
- Con Numba paralelo (`prange`) es donde mejor sale: ~0.01–0.016 s para 10^6 y ~0.15–0.17 s para 10^7. Es el más rápido de todos, seguramente porque paraleliza el bucle en código compilado sin tanta sobrecarga como `Pool`.

Conclusión: el original no escala, Numba secuencial ya acelera mucho, `Pool` escala razonablemente y Numba+`prange` es el que da mejores tiempos.